In [1]:
import pandas as pd
import numpy as np
import scipy.stats as scipy

In [2]:
df = pd.read_csv("./Data/E0_13.csv")
df_14 = pd.read_csv("./Data/E0_14.csv")

In [3]:
df.columns

Index([u'Div', u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR',
       u'HTHG', u'HTAG', u'HTR', u'Referee', u'HS', u'AS', u'HST', u'AST',
       u'HF', u'AF', u'HC', u'AC', u'HY', u'AY', u'HR', u'AR', u'B365H',
       u'B365D', u'B365A', u'BWH', u'BWD', u'BWA', u'IWH', u'IWD', u'IWA',
       u'LBH', u'LBD', u'LBA', u'PSH', u'PSD', u'PSA', u'WHH', u'WHD', u'WHA',
       u'SJH', u'SJD', u'SJA', u'VCH', u'VCD', u'VCA', u'Bb1X2', u'BbMxH',
       u'BbAvH', u'BbMxD', u'BbAvD', u'BbMxA', u'BbAvA', u'BbOU', u'BbMx>2.5',
       u'BbAv>2.5', u'BbMx<2.5', u'BbAv<2.5', u'BbAH', u'BbAHh', u'BbMxAHH',
       u'BbAvAHH', u'BbMxAHA', u'BbAvAHA', u'PSCH', u'PSCD', u'PSCA'],
      dtype='object')

In [4]:
res_13 = df.ix[:,:23]
res_13 = res_13.drop(['Div','Date','Referee'],axis=1)
res_14 = df_14.ix[:,:23]
res_14 = res_14.drop(['Div','Date','Referee'],axis=1)
table_features = df.ix[:,:7]
table_features = table_features.drop(['FTHG','FTAG','Div','Date'],axis=1)
bet_13 = df.ix[:,23:]


In [5]:
res_13.head()
feature_table = df.ix[:,:23]

In [6]:
#Team, Home Goals Score, Away Goals Score, Attack Strength, Home Goals Conceded, Away Goals Conceded, Defensive Strength
table_13 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS'))

In [7]:
avg_home_scored_13 = res_13.FTHG.sum() / 380.0
avg_away_scored_13 = res_13.FTAG.sum() / 380.0
avg_home_conceded_13 = avg_away_scored_13
avg_away_conceded_13 = avg_home_scored_13
print "Average number of goals at home",avg_home_scored_13
print "Average number of goals away", avg_away_scored_13
print "Average number of goals conceded at home",avg_away_conceded_13
print "Average number of goals conceded away",avg_home_conceded_13


Average number of goals at home 1.57368421053
Average number of goals away 1.19473684211
Average number of goals conceded at home 1.57368421053
Average number of goals conceded away 1.19473684211


In [8]:
res_home = res_13.groupby('HomeTeam')
res_away = res_13.groupby('AwayTeam')

In [9]:
list(res_home)[0]

('Arsenal',
     HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  HS  AS  HST  \
 0    Arsenal     Aston Villa     1     3   A     1     1   D  16   9    4   
 27   Arsenal       Tottenham     1     0   H     1     0   H  12  14    5   
 46   Arsenal           Stoke     3     1   H     2     1   H  16   9    8   
 70   Arsenal         Norwich     4     1   H     1     0   H  20  12   11   
 90   Arsenal       Liverpool     2     0   H     1     0   H  12  12    7   
 110  Arsenal     Southampton     2     0   H     1     0   H   9  10    4   
 131  Arsenal            Hull     2     0   H     1     0   H  20   7    7   
 147  Arsenal         Everton     1     1   D     0     0   D  11  12    5   
 169  Arsenal         Chelsea     0     0   D     0     0   D   7  13    2   
 190  Arsenal         Cardiff     2     0   H     0     0   D  28   8    6   
 210  Arsenal          Fulham     2     0   H     0     0   D  22   8    8   
 237  Arsenal  Crystal Palace     2     0   H     0 

In [10]:
table_13.Team = res_home.HomeTeam.all().values
table_13.HGS = res_home.FTHG.sum().values
table_13.HGC = res_home.FTAG.sum().values
table_13.AGS = res_away.FTAG.sum().values
table_13.AGC = res_away.FTHG.sum().values
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,36,32,NaN,NaN,11,30,NaN,NaN
1,Aston Villa,22,17,NaN,NaN,29,32,NaN,NaN
2,Cardiff,20,12,NaN,NaN,35,39,NaN,NaN
3,Chelsea,43,28,NaN,NaN,11,16,NaN,NaN
4,Crystal Palace,18,15,NaN,NaN,23,25,NaN,NaN


In [11]:
table_13.HAS = (table_13.HGS / 19.0) / avg_home_scored_13
table_13.AAS = (table_13.AGS / 19.0) / avg_away_scored_13
table_13.HDS = (table_13.HGC / 19.0) / avg_home_conceded_13
table_13.ADS = (table_13.AGC / 19.0) / avg_away_conceded_13
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,36,32,1.204013,1.409692,11,30,0.484581,1.003344
1,Aston Villa,22,17,0.735786,0.748899,29,32,1.277533,1.070234
2,Cardiff,20,12,0.668896,0.528634,35,39,1.541850,1.304348
3,Chelsea,43,28,1.438127,1.233480,11,16,0.484581,0.535117
4,Crystal Palace,18,15,0.602007,0.660793,23,25,1.013216,0.836120


In [12]:
feature_table = feature_table[['HomeTeam','AwayTeam','FTR']]
f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feature_table['HAS'] = f_HAS
feature_table['HDS'] = f_HDS
feature_table['AAS'] = f_AAS
feature_table['ADS'] = f_ADS

In [13]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HAS,HDS,AAS,ADS
0,Arsenal,Aston Villa,A,1.204013,0.484581,1.409692,1.003344
1,Liverpool,Stoke,H,1.772575,0.792952,2.114537,1.070234
2,Norwich,Everton,D,0.568562,0.792952,0.484581,1.471572
3,Sunderland,Fulham,A,0.702341,1.189427,0.881057,1.103679
4,Swansea,Man United,A,1.103679,1.145374,0.925110,0.936455


In [14]:
def transformResult(row):
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return -1
    else:
        return 0

In [15]:
feature_table["Result"] = feature_table.apply(lambda row: transformResult(row),axis=1)

In [16]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HAS,HDS,AAS,ADS,Result
0,Arsenal,Aston Villa,A,1.204013,0.484581,1.409692,1.003344,-1
1,Liverpool,Stoke,H,1.772575,0.792952,2.114537,1.070234,1
2,Norwich,Everton,D,0.568562,0.792952,0.484581,1.471572,0
3,Sunderland,Fulham,A,0.702341,1.189427,0.881057,1.103679,-1
4,Swansea,Man United,A,1.103679,1.145374,0.925110,0.936455,-1


In [17]:
X_train = feature_table[['HAS','HDS','AAS','ADS']]
y_train = feature_table['Result']

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [19]:
clf = DecisionTreeClassifier()
clf2.fit(X_train,y_train)
clf3 = XGBClassifier()
clf3.fit(X_train,y_train)

NameError: name 'clf2' is not defined

In [ ]:
y_pred = clf3.predict(X_train)
accuracy_score(y_pred,y_train)